**<font size = 5 color = black>GPU Device Install</font>**

In [1]:
import torch
import torchvision
from torch2trt import torch2trt

model = torchvision.models.resnet18(pretrained=False)

**<font size = 5 color = black>Add Resnet Module for Road Find And Avoidance</font>**

In [2]:
class MultiOutputModel(torch.nn.Module):
    def __init__(self):
        super( MultiOutputModel, self).__init__()
        self.out1 = torch.nn.Linear(512, 2)
        self.out2 = torch.nn.Linear(512, 2)
        
    def forward(self, x):
        x_out1 = self.out1(x)
        x_out2 = self.out2(x) 
        return x_out1, x_out2

In [ ]:
model.fc = MultiOutputModel()
model = model.cuda().eval().half()
model.load_state_dict(torch.load('Merge_Function_Robot_final_v13.pth'))

device = torch.device('cuda')

data = torch.zeros((1, 3, 224, 224)).cuda().half()
# model_merge = model
model_merge = torch2trt(model, [data], fp16_mode=True)
print(model_merge)

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

**<font size = 5 color = black>Essential Component</font>**

In [5]:
from jetbot import Robot, Camera, bgr8_to_jpeg
from IPython.display import display
import ipywidgets.widgets as widgets
import traitlets

**<font size = 5 color = black> Changable Parameters</font>**

In [6]:
robot = Robot()
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

speed_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

**<font color = black size = 5>Gotten Value</font>**

In [7]:
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

**<font size = 5 color = black>Road Find Executing</font>**

In [8]:
import torch.nn.functional as F
import time

angle = 0.0
angle_last = 0.0

y_road, y_avoid = 0, 0

def execute(change):
    global angle, angle_last, blocked_slider, robot
    image = change['new']
    y_road, y_avoid = model_merge(preprocess(image))
    
    # Classification
    y_avoid = F.softmax(y_avoid, dim=1)
    prob_blocked = float(y_avoid.flatten()[0])
    blocked_slider.value = prob_blocked
    
    if prob_blocked > 0.85:
        robot.stop()
    else:     
        #Regression
        xy = y_road.detach().float().cpu().numpy().flatten()
        x = xy[0] 
        y = (0.5 - xy[1]) / 2.0
        x_slider.value = x
        y_slider.value = y

        speed_slider.value = speed_gain_slider.value

        angle = np.arctan2(x, y)
        pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
        angle_last = angle

        steering_slider.value = pid + steering_bias_slider.value

        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

**<font size = 5 color = black>Avoidance Module Executing</font>**

**<font size = 5 color = purple> Control Interface<font>**

In [ ]:
display(widgets.HBox([
    widgets.VBox([widgets.HBox([image, blocked_slider,y_slider]), x_slider]), 
    widgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])
    ])
)

In [14]:
camera.observe(execute, names='value')  # this attaches the 'execute' function to the 'value' traitleT

**<font size = 5 color = purple> Stop Command<font>**

In [11]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()